In [1]:
import findspark

In [2]:
findspark.init('/home/vboxuser/spark-3.5.0-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('mylogreg').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/10 23:22:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.ml.classification import LogisticRegression

In [7]:
my_data = spark.read.format('libsvm').load('/home/vboxuser/Downloads/sample_libsvm_data.txt')

24/01/10 23:26:39 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [8]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [9]:
my_log_reg_model = LogisticRegression()

In [10]:
fitted_logreg = my_log_reg_model.fit(my_data)

24/01/10 23:29:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [11]:
log_summary = fitted_logreg.summary

In [12]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [13]:
lr_train,lr_test = my_data.randomSplit([0.7,0.3])

In [14]:
final_model = LogisticRegression()

In [15]:
fit_final = final_model.fit(lr_train)

In [16]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [17]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[98,99,100,1...|[32.0292171926834...|[0.99999999999998...|       0.0|
|  0.0|(692,[122,123,148...|[18.4397749219685...|[0.99999999018913...|       0.0|
|  0.0|(692,[123,124,125...|[29.6568748697297...|[0.99999999999986...|       0.0|
|  0.0|(692,[126,127,128...|[27.5968074060158...|[0.99999999999896...|       0.0|
|  0.0|(692,[126,127,128...|[36.2461099989588...|[0.99999999999999...|       0.0|
|  0.0|(692,[126,127,128...|[26.6442309258328...|[0.99999999999731...|       0.0|
|  0.0|(692,[127,128,129...|[24.9962587312099...|[0.99999999998606...|       0.0|
|  0.0|(692,[129,130,131...|[17.5051264955195...|[0.99999997501840...|       0.0|
|  0.0|(692,[151,152,153...|[31.7346761025578...|[0.99999999999998...|       0.0|
|  1.0|(692,[99,

In [21]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [22]:
my_eval = BinaryClassificationEvaluator()

In [23]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [24]:
my_final_roc

1.0